In [1]:
ROOT = '/home/anhphantq/otto'
max_session = 64


In [2]:
DATA_FOLDER = ROOT + '/splitted_data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')


In [2]:
DATA_FOLDER = ROOT + '/data'
import pandas as pd
test_df = pd.read_parquet(DATA_FOLDER + '/test.parquet')
train_df = pd.read_parquet(DATA_FOLDER + '/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
old_shape = train_df.shape[0]
old_num_aid = len(np.unique(train_df.aid))
train_df = train_df.drop(old_index.flatten())
num_aid = len(np.unique(train_df.aid))
train_df.shape[0], old_shape,  old_shape - train_df.shape[0], num_aid, old_num_aid, old_num_aid - num_aid

(163555218, 216716096, 53160878, 1839558, 1855603, 16045)

In [ ]:
train_df

In [ ]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/splitted_unidata_sub/train_v1.parquet')


In [ ]:
import pandas as pd
test_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/test_v1.parquet')
train_df = pd.read_parquet('/content/drive/MyDrive/ColabNotebooks/unidata_sub/train_v1.parquet')


#Add subsession

In [ ]:
import numpy as np

def add_diff(df):
  df = df.sort_values(['session', 'ts'])
  df['diff'] = df['ts'].diff()
  session = df['session'].values
  ends_idx = session[1: ] - session[:-1]
  start_session = np.argwhere(ends_idx > 0) + 1
  start_session  = np.vstack(([0], start_session))
  df.iloc[start_session, -1] = 0
  return df
  

In [ ]:
train_df = add_diff(train_df)
train_df

,session,aid,ts,type,diff
0,0,1517085,1659304800,0,0.0
1,0,1563459,1659304904,0,104.0
2,0,1309446,1659367439,0,62535.0
3,0,16246,1659367719,0,280.0
4,0,1781822,1659367871,0,152.0
...,...,...,...,...,...
163955175,11098523,175715,1661119197,0,0.0
163955176,11098524,1088524,1661119198,0,0.0
163955177,11098525,182927,1661119199,0,0.0
163955178,11098526,510055,1661119199,0,0.0


In [ ]:
test_df = add_diff(test_df)
test_df

,session,aid,ts,type,diff
0,11098528,11830,1661119200,0,0.0
1,11098529,1105029,1661119200,0,0.0
2,11098530,264500,1661119200,0,0.0
3,11098530,264500,1661119288,0,88.0
4,11098530,409236,1661119369,0,81.0
...,...,...,...,...,...
7683572,12899774,33035,1661723968,0,0.0
7683573,12899775,1743151,1661723970,0,0.0
7683574,12899776,548599,1661723972,0,0.0
7683575,12899777,384045,1661723976,0,0.0


In [ ]:
from tqdm import tqdm

def add_subsession(df, drop = False):
  cur_session = -1
  cur_subsession = -1
  subsessions = []
  max_afk = 0.5 * 3600

  count = 0

  remove = []
  for row in tqdm(df.itertuples(), total = df.shape[0]):
    if row.session != cur_session:

      if cur_subsession != -1 and cur_subsession >= max_session:
        session_tracking = np.array(session_tracking)
        m = np.max(session_tracking)
        upto_remove = m - max_session
        num_redundant = np.sum(np.where(session_tracking <= upto_remove, 1, 0))
        remove += np.arange(start_idx, start_idx + num_redundant).tolist()
      
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()

      #reset sesioon
      cur_session = row.session
      cur_subsession = 0
      
      start_idx = row[0]
      session_tracking = []

      #reset subsession
      sub_start_idx = row[0]
      count = 0 
      subsession_tracking = []





    if row.diff > max_afk:
      if count > 39:
        subsession_tracking = np.array(subsession_tracking)
        m = np.max(subsession_tracking)
        upto_remove = m - 9
        num_redundant = np.sum(np.where(subsession_tracking <= upto_remove, 1, 0))
        remove += np.arange(sub_start_idx, sub_start_idx + num_redundant).tolist()
      
      #reset subsession
      sub_start_idx = row[0]
      count = 0
      subsession_tracking = []
      cur_subsession += 1


    else:
      
      count += 1
    
    subsession_tracking.append(count)
    session_tracking.append(cur_subsession)
    subsessions.append(cur_subsession)



  df['subsession'] = subsessions
  if drop:
    print(f'Delete {len(remove)} rows')
    df = df.drop(index = remove, axis = 0)

  del subsessions
  return df

In [ ]:
train_df = add_subsession(train_df, drop = True)
train_df

100%|██████████| 163955180/163955180 [03:55<00:00, 695626.27it/s]


Delete 15255962 rows


,session,aid,ts,type,diff,subsession
0,0,1517085,1659304800,0,0.0,0
1,0,1563459,1659304904,0,104.0,0
2,0,1309446,1659367439,0,62535.0,1
3,0,16246,1659367719,0,280.0,1
4,0,1781822,1659367871,0,152.0,1
...,...,...,...,...,...,...
163955175,11098523,175715,1661119197,0,0.0,0
163955176,11098524,1088524,1661119198,0,0.0,0
163955177,11098525,182927,1661119199,0,0.0,0
163955178,11098526,510055,1661119199,0,0.0,0


In [ ]:
test_df = add_subsession(test_df, drop = False)
test_df

100%|██████████| 7683577/7683577 [00:07<00:00, 968975.44it/s]


,session,aid,ts,type,diff,subsession
0,11098528,11830,1661119200,0,0.0,0
1,11098529,1105029,1661119200,0,0.0,0
2,11098530,264500,1661119200,0,0.0,0
3,11098530,264500,1661119288,0,88.0,0
4,11098530,409236,1661119369,0,81.0,0
...,...,...,...,...,...,...
7683572,12899774,33035,1661723968,0,0.0,0
7683573,12899775,1743151,1661723970,0,0.0,0
7683574,12899776,548599,1661723972,0,0.0,0
7683575,12899777,384045,1661723976,0,0.0,0


# Save df

In [ ]:
train_df.to_parquet('/home/anhphantq/splitted_unidata_sub/train_v1.parquet', index = False)


In [ ]:
train_df.to_parquet('/home/anhphantq/unidata_sub/train_v1.parquet', index = False)

In [ ]:
test_df.to_parquet('/home/anhphantq/unidata_sub/test_v1.parquet', index = False)

In [ ]:
test_df.to_parquet('/home/anhphantq/splitted_unidata_sub/test_v1.parquet', index = False)

# align subsession to 0 -> 64

In [ ]:
def substract_redudant_session(df, test = False):
  min_sub = df.loc[:, ['session','subsession']].groupby('session').min()
  df = df.merge(min_sub, on = 'session', how = 'left')
  df['subsession'] = df['subsession_x'] - df['subsession_y']
  if not test:
    assert df['subsession'].max() == max_session - 1
  assert df['subsession'].min() == 0
  del df['subsession_x'], df['subsession_y']
  return df


In [ ]:
train_df = substract_redudant_session(train_df)
train_df

,session,aid,ts,type,diff,subsession
0,0,1517085,1659304800,0,0.0,0
1,0,1563459,1659304904,0,104.0,0
2,0,1309446,1659367439,0,62535.0,1
3,0,16246,1659367719,0,280.0,1
4,0,1781822,1659367871,0,152.0,1
...,...,...,...,...,...,...
148699844,11098523,175715,1661119197,0,0.0,0
148699845,11098524,1088524,1661119198,0,0.0,0
148699846,11098525,182927,1661119199,0,0.0,0
148699847,11098526,510055,1661119199,0,0.0,0


In [ ]:
test_df = substract_redudant_session(test_df, test = True)
test_df

,session,aid,ts,type,diff,subsession
0,11098528,11830,1661119200,0,0.0,0
1,11098529,1105029,1661119200,0,0.0,0
2,11098530,264500,1661119200,0,0.0,0
3,11098530,264500,1661119288,0,88.0,0
4,11098530,409236,1661119369,0,81.0,0
...,...,...,...,...,...,...
7683572,12899774,33035,1661723968,0,0.0,0
7683573,12899775,1743151,1661723970,0,0.0,0
7683574,12899776,548599,1661723972,0,0.0,0
7683575,12899777,384045,1661723976,0,0.0,0


# save vprops

In [ ]:
def vprops(df, path):
  vprop = df.loc[:, ['session','subsession', 'ts']].groupby('session').agg({
      'ts': ['min', 'max', 'count'],
      'subsession': 'max'
  })
  vprop['consistency'] = vprop.iloc[:, 1] - vprop.iloc[:, 0]
  vprop.reset_index().to_parquet(path, index = False)


In [ ]:
vprops(train_df, '/home/anhphantq/splitted_unidata_sub/train_vprops_v1.parquet')

In [ ]:
vprops(test_df, '/home/anhphantq/splitted_unidata_sub/test_vprops_v1.parquet')

In [ ]:
vprops(train_df, '/home/anhphantq/unidata_sub/train_vprops_v1.parquet')

In [ ]:
vprops(test_df, '/home/anhphantq/unidata_sub/test_vprops_v1.parquet')

# transform

In [ ]:
import numpy as np
from tqdm import tqdm
TRAIN_TS_MIN = train_df.ts.min()
def transform_df(df):
  df['ts'] = df['ts'] - TRAIN_TS_MIN
  df['clicks'] = np.where(df['type'] == 0, 1, 0)
  df['carts'] = np.where(df['type'] == 1, 1, 0)
  df['orders'] = np.where(df['type'] == 2, 1, 0)
      
  assert df['clicks'].sum() + df['carts'].sum() + df['orders'].sum() == df.shape[0]
  del df['type'], df['diff']




In [ ]:
transform_df(train_df)
train_df

,session,aid,ts,subsession,clicks,carts,orders
0,0,1517085,0,0,1,0,0
1,0,1563459,104,0,1,0,0
2,0,1309446,62639,1,1,0,0
3,0,16246,62919,1,1,0,0
4,0,1781822,63071,1,1,0,0
...,...,...,...,...,...,...,...
148699844,11098523,175715,1814397,0,1,0,0
148699845,11098524,1088524,1814398,0,1,0,0
148699846,11098525,182927,1814399,0,1,0,0
148699847,11098526,510055,1814399,0,1,0,0


In [ ]:
transform_df(test_df)
test_df

,session,aid,ts,subsession,clicks,carts,orders
0,11098528,11830,1814400,0,1,0,0
1,11098529,1105029,1814400,0,1,0,0
2,11098530,264500,1814400,0,1,0,0
3,11098530,264500,1814488,0,1,0,0
4,11098530,409236,1814569,0,1,0,0
...,...,...,...,...,...,...,...
7683572,12899774,33035,2419168,0,1,0,0
7683573,12899775,1743151,2419170,0,1,0,0
7683574,12899776,548599,2419172,0,1,0,0
7683575,12899777,384045,2419176,0,1,0,0


# Group to edges

In [ ]:

def encode(x):
  return np.sum(2 ** np.unique(x))

In [ ]:
from tqdm import tqdm
import numpy as np
step = train_df.shape[0] // 10
start = [i * step for i in range(10)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

for i in tqdm(range(len(start))):

  train_df.iloc[start[i] : end[i]].groupby(['session', 'aid']).agg({ 
      'ts': ['last','first'],
      'clicks': 'sum',
      'carts': 'sum',
      'orders': 'sum',
      'subsession': encode
  }).reset_index().to_parquet(f'/home/anhphantq/unidata_sub/edges_sub_frag_{i}_v1.parquet', index = False)

100%|██████████| 10/10 [1:17:08<00:00, 462.90s/it]


In [ ]:
from tqdm import tqdm
import numpy as np
step = train_df.shape[0] // 10
start = [i * step for i in range(10)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

for i in tqdm(range(len(start))):

  train_df.iloc[start[i] : end[i]].groupby(['session', 'aid']).agg({ 
      'ts': ['last','first'],
      'clicks': 'sum',
      'carts': 'sum',
      'orders': 'sum',
      'subsession': encode
  }).reset_index().to_parquet(f'/home/anhphantq/splitted_unidata_sub/edges_sub_frag_{i}_v1.parquet', index = False)

100%|██████████| 10/10 [1:16:46<00:00, 460.65s/it]


In [ ]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/home/anhphantq/unidata_sub/edges_test_sub_v1.parquet', index = False)

In [ ]:
import numpy as np
test_df.groupby(['session', 'aid']).agg({ 
    'ts': ['last','first'],
    'clicks': 'sum',
    'carts': 'sum',
    'orders': 'sum',
    'subsession': encode
}).reset_index().to_parquet('/home/anhphantq/splitted_unidata_sub/edges_test_sub_v1.parquet', index = False)

RECENT FEATURES

In [3]:
import numpy as np
def recent_7days(df):
    unique_aids = np.unique(df.aid.values)
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear
    
    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6
    
    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]
    
    df['dayofyear_type'] = df.dayofyear * 10 + df.type
    item_agg = pd.DataFrame(df[['aid', 'dayofyear_type', 'ts']].groupby(['aid', 'dayofyear_type'])['ts'].count()).rename(columns = {'ts': 'count'})


    dates = pd.DataFrame({'dayofyear': np.arange(dayofyear_min, dayofyear_max + 1), 'tmp': 1})
    types = pd.DataFrame({'types': [0,1,2], 'tmp' : 1})
    aids = pd.DataFrame({'aid': np.unique(df.aid), 'tmp': 1})
    dates = dates.merge(types, on = 'tmp')

    dates = aids.merge(dates, on = 'tmp')
    del dates['tmp']
    dates['dayofyear_type'] = dates['dayofyear'] * 10 + dates['types']
    del dates['types']
    dates['count'] = np.nan
    dates = dates.groupby(['aid', 'dayofyear_type']).first()


    cols = []
    for i in range(7,0,-1):
        for j in range(3):
            cols.append(f'recent_day{i}_type{j}')
 
    
    item_agg = dates.combine_first(item_agg).fillna(0).reset_index()
    item_agg['count'] = item_agg['count'].values.astype(int)
    recent_7days = pd.DataFrame(data = item_agg[['aid', 'count']].groupby('aid')['count'].apply(list).tolist(), columns = [col for col in cols])
    
    
    item_agg = pd.concat((pd.DataFrame({
        'aid': np.unique(item_agg.aid)
    }), recent_7days), axis = 1)
    
    
    item_agg = pd.DataFrame({'aid': unique_aids, **dict(
        [(col, np.nan) for col in cols], 
    )}).set_index('aid').combine_first(item_agg.set_index('aid')).fillna(0)
    return item_agg.reset_index()

In [ ]:
item_agg = recent_7days(train_df)

/tmp/ipykernel_220630/1404067359.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofyear_type'] = df.dayofyear * 10 + df.type


In [ ]:
item_agg

,aid,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,...,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2
0,0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,206.0,18.0,5.0,99.0,11.0,0.0,13.0,0.0,0.0,...,3.0,211.0,27.0,6.0,228.0,19.0,2.0,335.0,32.0,7.0
4,4,8.0,1.0,0.0,12.0,1.0,0.0,7.0,0.0,0.0,...,0.0,8.0,1.0,0.0,10.0,0.0,0.0,14.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839553,1855598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1839554,1855599,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1839555,1855600,2.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,...,0.0,4.0,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
1839556,1855601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_agg.to_parquet("/home/anhphantq/unidata_sub/item_props_v1.parquet", index = False)

In [ ]:
item_agg.to_parquet("/home/anhphantq/splitted_unidata_sub/item_props_v1.parquet", index = False)

ITEM GLOBAL FEATURES

In [ ]:
TRAIN_TS_MIN = train_df.ts.min()

train_df

,session,aid,ts,type,date,dayofyear
63,0,1629651,1659964140,0,2022-08-08 13:09:00,220
64,0,1768568,1660015113,0,2022-08-09 03:18:33,221
65,0,1318324,1660015128,0,2022-08-09 03:18:48,221
66,0,1840418,1660024261,0,2022-08-09 05:51:01,221
67,0,1813509,1660024355,0,2022-08-09 05:52:35,221
...,...,...,...,...,...,...
216716091,12899776,1737908,1661723987,0,2022-08-28 21:59:47,240
216716092,12899777,384045,1661723976,0,2022-08-28 21:59:36,240
216716093,12899777,384045,1661723986,0,2022-08-28 21:59:46,240
216716094,12899778,561560,1661723983,0,2022-08-28 21:59:43,240


In [ ]:
def global_item_features(df):
    df['scaled_ts'] = (df.ts - TRAIN_TS_MIN) / 3600 / 24 
    max_ts = df.scaled_ts.max()
    df['time_decay']  =  0.9 ** (max_ts - df.scaled_ts )
    df['click'] = np.where(df['type'] == 0, df.time_decay, 0)  
    df['cart'] = np.where(df['type'] == 1, df.time_decay, 0)
    df['order'] = np.where(df['type'] == 2, df.time_decay, 0)
    return df[['aid','scaled_ts', 'time_decay', 'click', 'cart', 'order']].groupby('aid').agg({
    'scaled_ts': ['max', 'min'], 
    'time_decay': 'sum', 
    'click': 'sum',
    'cart' : 'sum',
    'order': 'sum',
}).reset_index()

In [ ]:
item_glob = global_item_features(train_df)

In [ ]:
item_glob

aid  scaled_ts             time_decay        click        cart  \
                        max       min          sum          sum         sum   
0              0  20.863345  1.452789    13.572334    13.572334    0.000000   
1              1  17.508866  2.738449     7.465299     7.319284    0.146015   
2              2  19.611991  4.524248     6.230244     6.230244    0.000000   
3              3  20.984317  0.434826  1461.256950  1322.898875  115.314236   
4              4  20.898519  0.388796    79.325614    75.313527    4.012088   
...          ...        ...       ...          ...          ...         ...   
1839553  1855598  17.944850  0.741273     2.034551     2.034551    0.000000   
1839554  1855599  19.679792  3.639051     4.168844     4.168844    0.000000   
1839555  1855600  20.814815  1.492014    27.600753    25.150745    2.321962   
1839556  1855601  12.285602  0.274120     7.111232     6.075555    1.035678   
1839557  1855602  20.750035  3.716285     7.147244     7.147244    0.000000   

             order  
               sum  
0         0.000000  
1         0.000000  
2         0.000000  
3        23.043839  
4         0.000000  
...            ...  
1839553   0.000000  
1839554   0.000000  
1839555   0.128045  
1839556   0.000000  
1839557   0.000000  

[1839558 rows x 7 columns]

In [ ]:
item_glob.to_parquet("/home/anhphantq/unidata_sub/item_glob_v1.parquet", index = False)

In [ ]:
item_glob.to_parquet("/home/anhphantq/splitted_unidata_sub/item_glob_v1.parquet", index = False)

RECENT INTERACT FEATURES

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import numpy as np
def recent_7days_inter(args):
    df, batchname = args
    unique_aids = np.unique(df.aid.values)
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear

    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6

    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]

    df['dayofyear_type'] = df.dayofyear * 10 + df.type
    item_agg = pd.DataFrame(df[['aid', 'dayofyear_type', 'ts']].groupby(['aid', 'dayofyear_type'])['ts'].count()).rename(columns = {'ts': 'count'})


    dates = pd.DataFrame({'dayofyear': np.arange(dayofyear_min, dayofyear_max + 1), 'tmp': 1})
    types = pd.DataFrame({'types': [0,1,2], 'tmp' : 1})
    dates = dates.merge(types, on = 'tmp')
    unique_user_aid = df[['session', 'aid']].groupby(['session', 'aid']).first().reset_index()
    unique_user_aid['tmp'] = 1
    dates = unique_user_aid.merge(dates, on = 'tmp')
    del dates['tmp']

    dates ['session_dayofyear_type'] = (dates.dayofyear * 10 + dates.types) * 100000000 + dates.session
    del dates['dayofyear'], dates['types'], dates['session']
    dates = dates.groupby(['aid', 'session_dayofyear_type']).first()
    dates['count'] = np.nan
    df['session_dayofyear_type'] =  df.dayofyear_type * 100000000 + df.session
    df = df[['session_dayofyear_type','aid', 'ts']].groupby(['session_dayofyear_type', 'aid']).count().rename(columns = {'ts': 'count'})

    cols = []
    for i in range(7,0,-1):
        for j in range(3):
            cols.append(f'recent_inter_day{i}_type{j}')
            
    item_agg = dates.combine_first(df).fillna(0).reset_index()
    item_agg['count'] = item_agg['count'].values.astype(int)
    item_agg['session'] = item_agg['session_dayofyear_type'] % 100000000
    
    recent_7days = pd.DataFrame(data = item_agg[['session', 'aid', 'count']].groupby(['session', 'aid'])['count'].apply(list).tolist(), columns = [col for col in cols])
    index = item_agg[['session', 'aid']].groupby(['session', 'aid']).first()
    recent_7days = pd.concat((index.reset_index(), recent_7days), axis = 1).set_index(['session', 'aid']).sort_index()
    return recent_7days, batchname

In [5]:
from tqdm import tqdm
import numpy as np

step = test_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = test_df.shape[0] 



In [6]:
batches = [test_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]
batch_names = [i for i in range(len(batches))]


In [7]:
del test_df

In [8]:
import multiprocessing
import os
assert os.path.exists('/home/anhphantq/splitted_unidata_sub/interact')
pool = multiprocessing.Pool(64)
for rs, batch in tqdm(pool.imap_unordered(recent_7days_inter,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
  rs.to_parquet(f'/home/anhphantq/splitted_unidata_sub/interact/recent_interacts_frag_{batch}_v1.parquet', index = True)
  
pool.close()
pool.join()

100%|██████████| 64/64 [00:23<00:00,  2.67it/s]


In [8]:
import multiprocessing
import os
assert os.path.exists('/home/anhphantq/unidata_sub/interact')
pool = multiprocessing.Pool(64)
for rs, batch in tqdm(pool.imap_unordered(recent_7days_inter,  tuple(zip(batches, batch_names)), chunksize = 1), total = len(batches)):
  rs.to_parquet(f'/home/anhphantq/unidata_sub/interact/recent_interacts_frag_{batch}_v1.parquet', index = True)
  
pool.close()
pool.join()

100%|██████████| 64/64 [00:21<00:00,  2.94it/s]
